In [1]:
!pip install flask_sqlalchemy
!pip install flask_login
!pip install flask_user

!pip install pandas
!pip install jupyter-dash

!pip install email_validator

!pip install wtforms

In [2]:
from wtforms import Form, StringField, PasswordField, validators, ValidationError
from wtforms.validators import InputRequired, Email, email_validator
#from wtforms.fields.html5 import EmailField
from wtforms.widgets import TextArea

In [3]:
from flask import Flask, render_template_string, Blueprint

from flask import g, request, redirect, url_for


from flask_sqlalchemy import SQLAlchemy

from flask_login import LoginManager, login_user

from flask_user import login_required, UserManager, UserMixin

from functools import wraps

from jupyter_dash import JupyterDash

import dash
from dash import dcc
from dash import html
import pandas as pd


In [4]:
# This file contains an example Flask-User application.
# To keep the example simple, we are applying some unusual techniques:
# - Placing everything in one file
# - Using class-based configuration (instead of file-based configuration)
# - Using string-based templates (instead of file-based templates)

from flask import Flask, render_template_string
from flask_sqlalchemy import SQLAlchemy
from flask_user import login_required, UserManager, UserMixin

In [5]:
import os
cwd = os.getcwd()
cwd

'/home/andrewcgaitskell/Documents/Code/flask-dance-github/flask-login-dash'

In [6]:
# Class-based application configuration
class ConfigClass(object):
    """ Flask application config """

    # Flask settings
    SECRET_KEY = 'thisissecret'

    #server.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + cwd + '/login.db'
    #server.config['SECRET_KEY'] = 'thisissecret'
    
    # Flask-SQLAlchemy settings
    SQLALCHEMY_DATABASE_URI = 'sqlite:///' + cwd + '/login.db'    # File-based SQL database
    SQLALCHEMY_TRACK_MODIFICATIONS = False    # Avoids SQLAlchemy warning

    # Flask-User settings
    USER_APP_NAME = "Flask-User QuickStart App"      # Shown in and email templates and page footers
    USER_ENABLE_EMAIL = False      # Disable email authentication
    USER_ENABLE_USERNAME = True    # Enable username authentication
    USER_REQUIRE_RETYPE_PASSWORD = False    # Simplify register form


#app_flask = Flask(__name__)

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

#app_dash = JupyterDash(__name__, external_stylesheets=external_stylesheets, url_base_pathname='/dash/')

# Create server variable with Flask server object for use with gunicorn
server = app_dash.server


In [7]:


def create_app():
    """ Flask application factory """
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
    # Create Flask app load app.config
    app = JupyterDash(__name__, serve_locally = False, external_stylesheets=external_stylesheets, url_base_pathname='/dash/')
    server = app.server
    
    server.config.from_object(__name__+'.ConfigClass')

    # Initialize Flask-SQLAlchemy
    db = SQLAlchemy(server)

    # Define the User data-model.
    # NB: Make sure to add flask_user UserMixin !!!
    class User(db.Model, UserMixin):
        __tablename__ = 'users'
        id = db.Column(db.Integer, primary_key=True)
        active = db.Column('is_active', db.Boolean(), nullable=False, server_default='1')

        # User authentication information. The collation='NOCASE' is required
        # to search case insensitively when USER_IFIND_MODE is 'nocase_collation'.
        username = db.Column(db.String(100, collation='NOCASE'), nullable=False, unique=True)
        password = db.Column(db.String(255), nullable=False, server_default='')
        email_confirmed_at = db.Column(db.DateTime())

        # User information
        first_name = db.Column(db.String(100, collation='NOCASE'), nullable=False, server_default='')
        last_name = db.Column(db.String(100, collation='NOCASE'), nullable=False, server_default='')

    # Create all database tables
    db.create_all()

    # Setup Flask-User and specify the User data-model
    user_manager = UserManager(server, db, User)

    # The Home page is accessible to anyone
    @server.route('/')
    def home_page():
        # String-based templates
        return render_template_string("""
            {% extends "flask_user_layout.html" %}
            {% block content %}
                <h2>Home page</h2>
                <p><a href={{ url_for('user.register') }}>Register</a></p>
                <p><a href={{ url_for('user.login') }}>Sign in</a></p>
                <p><a href={{ url_for('home_page') }}>Home page</a> (accessible to anyone)</p>
                <p><a href={{ url_for('member_page') }}>Member page</a> (login required)</p>
                <p><a href={{ url_for('user.logout') }}>Sign out</a></p>
            {% endblock %}
            """)

    # The Members page is only accessible to authenticated users via the @login_required decorator
    @server.route('/members')
    @login_required    # User must be authenticated
    def member_page():
        # String-based templates
        return render_template_string("""
            {% extends "flask_user_layout.html" %}
            {% block content %}
                <h2>Members page</h2>
                <p><a href={{ url_for('user.register') }}>Register</a></p>
                <p><a href={{ url_for('user.login') }}>Sign in</a></p>
                <p><a href={{ url_for('home_page') }}>Home page</a> (accessible to anyone)</p>
                <p><a href={{ url_for('member_page') }}>Member page</a> (login required)</p>
                <p><a href={{ url_for('user.logout') }}>Sign out</a></p>
            {% endblock %}
            """)

    return app, server



In [8]:
#login_manager = LoginManager()
#login_manager.init_app(server)



@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

@server.route('/')
def index():
    user = User.query.filter_by(username='admin').first()
    login_user(user)
    return 'You are now logged in!'

@server.route('/logout')
@login_required
def logout():
    logout_user()
    return 'You are now logged out!'

@server.route('/home')
@login_required
def home():
    return 'The current FLASK user is ' + current_user.username


In [9]:
app, server = create_app()

In [10]:
def protect_dashviews(dash_app):
    for view_func in dash_app.server.view_functions:
        if view_func.startswith(dash_app.config.url_base_pathname):
            dash_app.server.view_functions[view_func] = login_required(dash_app.server.view_functions[view_func])

In [11]:

# TODO how to add login_required for dash? 
#app_dash = Dash(server=app_flask, url_base_pathname='/dash/')
##app.layout = html.H1('MY DASH APP')


app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for your data.
    '''),
])


In [12]:
#server.view_function

In [13]:
#app.run_server(debug=True)

In [14]:
protect_dashviews(app)

In [ ]:
app.run(host='localhost', port=5000)##, debug=True)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5000 (Press CTRL+C to quit)
127.0.0.1 - - [06/Jun/2022 18:26:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2022 18:26:22] "GET /dash/ HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2022 18:26:24] "GET /dash/_favicon.ico?v=2.4.1 HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2022 18:26:24] "GET /dash/_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2022 18:26:24] "GET /dash/_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2022 18:26:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2022 18:26:35] "GET /user/sign-out HTTP/1.1" 302 -
127.0.0.1 - - [06/Jun/2022 18:26:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2022 18:26:44] "GET /dash/ HTTP/1.1" 302 -
127.0.0.1 - - [06/Jun/2022 18:26:44] "GET /user/sign-in?next=/dash/ HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2022 18:26:53] "POST /user/sign-in?next=/dash/ HTTP/1.1" 302 -
127.0.0.1 - - [06/Jun/2022 18:26:53] "GET /dash/ HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2022 18:26:54] "GET /dash/_dash-layout HTTP/1.1" 200 -
12